In [ ]:
%pip install -q ultralytics

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov10s.pt')

In [ ]:
import cv2
image_bgr = cv2.imread("car-in-front.jpg")

In [ ]:
import matplotlib.pyplot as plt
from copy import deepcopy

frame = deepcopy(image_bgr)
prediction = model.predict(frame)
for result in prediction[0].boxes:
    x1, y1, x2, y2 = result.xyxy[0].cpu().numpy()  # Move tensor to CPU before converting to NumPy
    model_class = int(result.cls[0].cpu().numpy())
    if model_class in {2, 3, 5, 7}:
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 10)
image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

# Display the image using Matplotlib
plt.imshow(image_rgb)
plt.title("Displayed Image")
plt.axis('off')  # Hide the axis ticks and labels
plt.show()

In [ ]:
from os import getenv

model_version = '1'

deployed_model_name = f"distance-{model_version}"
infer_endpoint = f"https://distance-{model_version}-distance-prediction{getenv("BASE_URL")}"
infer_url = f"{infer_endpoint}/v2/models/{deployed_model_name}/infer"
print(infer_url)

In [ ]:
import requests

def rest_request(data):
    json_data = {
        "inputs": [
            {
                "name": "dense_input",
                "shape": [1, 6],
                "datatype": "FP32",
                "data": data
            }
        ]
    }

    headers = { "Authorization": f"Bearer {getenv("TOKEN")}" }
    response = requests.post(infer_url, json=json_data, verify=False, headers=headers)
    response_dict = response.json()
    return response_dict['outputs'][0]['data']

In [ ]:
data = [336.609784, 497.959391, 543.540077, 526.678117, 19, 41.264181]
prediction = rest_request(data)
print(prediction[0])
if prediction[0] > 0.5:
    print("Car rapidly approaching, braking hard!")